In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader

# Load the dataset

In [6]:
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv", index_col=0)

In [7]:
train.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
id,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


# Process the dataset to remove null

In [8]:
train["prompt"] = train.prompt.map(lambda x: eval(x)[0])
train["response_a"] = train.response_a.map(lambda x: eval(x.replace("null","''"))[0])
train["response_b"] = train.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Generate categorical data

In [9]:
train["class_name"] = train[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
train["class_label"] = train.class_name.map({'winner_model_a':0, 'winner_model_b': 1, 'winner_tie' : 2})


train.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label
id,,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1
65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1,winner_tie,2
96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0,winner_model_a,0
198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0,winner_model_b,1


# Make pair by joining the prompt and response

In [10]:
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except Exception:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except Exception:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except Exception:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",
                      f"Prompt: {prompt}\n\nResponse: {response_b}"
                     ]
    return row

In [11]:
train = train.apply(make_pairs, axis=1)  
train.head(2)

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label,encode_fail,options
id,,,,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0,False,[Prompt: Is it morally right to try to have a ...
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1,False,[Prompt: What is the difference between marria...


# Drop failed UTF-8 checking

In [12]:
print(train.encode_fail.value_counts(normalize=False))
train = train[train['encode_fail'] == False]
print(train.encode_fail.value_counts(normalize=False))


encode_fail
False    56885
True       592
Name: count, dtype: int64
encode_fail
False    56885
Name: count, dtype: int64


# Remove unwanted columns

In [13]:
from sklearn.model_selection import train_test_split  # Import package

train = train.drop(['model_a', 'model_b', 'prompt', 'response_a','response_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'class_name'], axis=1)

train_df, valid_df = train_test_split(train, test_size=0.2, stratify=train["class_label"])

In [14]:
train_df.shape

(45508, 3)

# Make match dataloader

In [15]:
from torch.utils.data import Dataset

class TextClassificationDataset(Dataset):
    def __init__(self, dataSet: pd.DataFrame, tokenizer):
        self.labels = dataSet['class_label'].values
        self.values = dataSet['options'].values
        self.dataSet = dataSet
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        text = self.values[index]
        label = self.labels[index]
        encoding = self.tokenizer(
            text,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = torch.reshape(encoding['input_ids'], (-1, ))
        attention_mask = torch.reshape(encoding['attention_mask'], (-1, ))
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load Models for tokeniser and classification from Hugging face

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
model

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-05-18 14:14:38.469037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747577678.678407      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747577678.733327      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [17]:
train_dataSet = TextClassificationDataset(train_df, tokenizer)
valid_dataSet = TextClassificationDataset(valid_df, tokenizer)

batch_size = 64
train_loader = DataLoader(train_dataSet, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataSet, batch_size=batch_size)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
!nvidia-smi

In [20]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), 2e-5)

# Training and eval loop

In [21]:
num_epochs = 4
learning_rate = 2e-5

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    epoch_loss = 0
    model.train()

    for i, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        print(f" {i + 1}/{len(train_loader)}")
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Average Training Loss: {avg_epoch_loss:.4f}")

    valid_err = 0
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(valid_loader):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            valid_err += loss.item()
            print(f" {i + 1}/{len(valid_loader)}")
    avg_valid_loss = valid_err / len(valid_loader)
    print(f"Average Validation Loss: {avg_valid_loss:.4f}")



Epoch 1/4


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 293.12 MiB is free. Process 3155 has 15.60 GiB memory in use. Of the allocated memory 15.28 GiB is allocated by PyTorch, and 33.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Save the model

In [ ]:
torch.save(model.state_dict(), "model.pt")
torch.save(model, "model.h5")